In [1]:
import os
import requests
import pandas as pd
import pickle
import sys


* Obtención de polígono en Monte Hermoso generado por QGIS

* Sería conveniente generar un script para tomar la geometría de un shape file o de cualquier dato vectorial en proyección EPSG:4326

In [2]:
# Definición de carpeta donde se encuentra dato vectorial
vec_folder = r'../Vectores/csv'
vec_file = r'geom_monte_h.csv'
print(os.path.exists(vec_folder))
df_geom = pd.read_csv(os.path.join(vec_folder,vec_file))
display(type(df_geom.loc[0]['wkt']))
geom = df_geom.loc[0]['wkt']
# Geometría generada por QGIS en wkt
print(geom)

# Modificaciones necesarias para que funcione la búsqueda en plataforma de la ESA
geom_2 = geom.replace('MultiPolygon','POLYGON')
geom_2 = geom_2.replace('(((','((')
geom_2 = geom_2.replace(')))','))')
print(geom_2)

True


str

MultiPolygon (((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316)))
POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))


### Armado de string de búsqueda para requerir a servidor de la ESA

In [8]:
filter_Sent = "contains(Name,'MSIL2A')"
url = 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
orderby = "ContentDate/Start"
top = "100"

initial_date_new = "gt 2022-03-03T00:00:00.000Z"
final_date_new = "lt 2022-05-03T00:00:00.000Z"
prefix_place =f"OData.CSC.Intersects(area=geography'SRID=4326;{geom_2}')"
renew_query_mis_pl = f"{url}?$filter={filter_Sent} and {prefix_place} and ContentDate/Start {initial_date_new} and ContentDate/Start {final_date_new}&$orderby={orderby}&$top={top}"
cloud_cover = str(10) # Se encuentra en procentaje pero hay que esscribirlo tipo string
prefix_cloud = f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {cloud_cover})"
prefix_dates = f"ContentDate/Start {initial_date_new} and ContentDate/Start {final_date_new}"
renew_query_mis_pl_cc = f"{url}?$filter={filter_Sent} and {prefix_place} and {prefix_cloud} and {prefix_dates}&$orderby={orderby}&$top={top}"
# print(prefix_place, renew_query_mis_pl, renew_query_mis_pl_cc, sep='\n')
# Query filtrando con 'contains' de MSIL2A
# Filter by mission
plat_str = 'SENTINEL-2'
# mission_filter = f"Collection/Name%20eq%20%27{plat_str}%27%20"
mission_filter = f"Collection/Name eq '{plat_str}'"
query_MSIL2A = f"{url}?$filter={mission_filter} and {filter_Sent} and {prefix_place} and {prefix_cloud} and {prefix_dates}&$orderby={orderby}&$top={top}"
req_list = [renew_query_mis_pl, renew_query_mis_pl_cc, query_MSIL2A]
display(req_list)

["https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100",
 "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10) and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$or

In [9]:
df_list = []
for i,req in enumerate(req_list):
    # print(req)
    json = requests.get(req).json()
    display(req)
    df_list.append(pd.DataFrame.from_dict(json['value']))
    display(df_list[i])

"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100"

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,28e6ae21-0310-4283-b1ed-efa8391301d3,S2B_MSIL2A_20220304T135109_N0510_R024_T20HPB_2...,application/octet-stream,856118643,2024-09-21T01:14:18.739000Z,2024-12-13T16:37:53.807406Z,2024-12-13T16:38:04.266684Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/04/S2...,"[{'Value': 'ab0092d5a2d32ff9f3fe7dd1bd598299',...","{'Start': '2022-03-04T13:51:09.000000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847718199 -3...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540000Z,2022-07-01T15:39:54.438282Z,2022-07-01T15:39:54.438282Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
2,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269000Z,2022-03-27T15:07:17.153349Z,2022-03-27T15:07:17.153349Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
3,application/octet-stream,3db1829f-7309-5c51-810f-20ffc7410946,S2A_MSIL2A_20220309T135121_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-09T21:38:02.833000Z,2022-03-27T01:19:12.338606Z,2022-03-27T01:19:12.338606Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/09/S2A_MSIL...,[],"{'Start': '2022-03-09T13:51:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.554108 -39.6...,"{'type': 'Polygon', 'coordinates': [[[-60.5541..."
4,application/octet-stream,e342880a-a6cf-4342-91e8-f4da4fc58399,S2A_MSIL2A_20220309T135121_N0510_R024_T20HPB_2...,application/octet-stream,683797174,2024-10-18T14:52:45.185000Z,2024-11-24T08:01:32.973193Z,2024-11-24T08:01:42.811846Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/09/S2...,"[{'Value': '0e1aece89f83faf4fbe010a207611bdd',...","{'Start': '2022-03-09T13:51:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.554109985236...,"{'type': 'Polygon', 'coordinates': [[[-60.5541..."
5,application/octet-stream,c1be4316-de04-5902-b0ac-b01b32aec350,S2A_MSIL2A_20220312T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-12T23:14:08.829000Z,2022-03-26T01:57:48.785924Z,2022-03-26T01:57:48.785924Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/12/S2A_MSIL...,[],"{'Start': '2022-03-12T14:01:01.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.008057 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0080..."
6,application/octet-stream,dbe0d837-0443-51da-b500-dbc1243df17c,S2B_MSIL2A_20220314T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-14T23:53:25.866000Z,2022-03-27T03:15:16.711463Z,2022-03-27T03:15:16.711463Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/14/S2B_MSIL...,[],"{'Start': '2022-03-14T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
7,application/octet-stream,e1973e4f-cffa-5441-965b-a4c8862d16e8,S2B_MSIL2A_20220317T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-17T23:19:50.743000Z,2022-03-29T03:39:33.894231Z,2022-03-29T03:39:33.894231Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/17/S2B_MSIL...,[],"{'Start': '2022-03-17T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.005646 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0

"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10) and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100"

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,28e6ae21-0310-4283-b1ed-efa8391301d3,S2B_MSIL2A_20220304T135109_N0510_R024_T20HPB_2...,application/octet-stream,856118643,2024-09-21T01:14:18.739000Z,2024-12-13T16:37:53.807406Z,2024-12-13T16:38:04.266684Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/04/S2...,"[{'Value': 'ab0092d5a2d32ff9f3fe7dd1bd598299',...","{'Start': '2022-03-04T13:51:09.000000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847718199 -3...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540000Z,2022-07-01T15:39:54.438282Z,2022-07-01T15:39:54.438282Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
2,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269000Z,2022-03-27T15:07:17.153349Z,2022-03-27T15:07:17.153349Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
3,application/octet-stream,c1be4316-de04-5902-b0ac-b01b32aec350,S2A_MSIL2A_20220312T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-12T23:14:08.829000Z,2022-03-26T01:57:48.785924Z,2022-03-26T01:57:48.785924Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/12/S2A_MSIL...,[],"{'Start': '2022-03-12T14:01:01.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.008057 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0080..."
4,application/octet-stream,41bd4637-3ca3-5ed3-bac7-6b5b66f6158e,S2A_MSIL2A_20220322T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-23T00:20:11.063000Z,2022-03-28T04:12:07.993475Z,2022-03-28T04:12:07.993475Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/22/S2A_MSIL...,[],"{'Start': '2022-03-22T14:01:01.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.002075 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0020..."
5,application/octet-stream,9ce5ef04-d1c3-5f9c-ae44-f1970d09e5e5,S2B_MSIL2A_20220327T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-27T19:15:06.268000Z,2022-03-27T19:20:05.514592Z,2022-03-27T19:20:05.514592Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/27/S2B_MSIL...,[],"{'Start': '2022-03-27T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.99878 -38.82...,"{'type': 'Polygon', 'coordinates': [[[-60.9987..."
6,application/octet-stream,d47474a5-b1b9-4824-bfb7-66b1ddfb91e1,S2B_MSIL2A_20220327T140049_N0510_R067_T20HPB_2...,application/octet-stream,528857225,2024-10-19T13:27:21.088000Z,2024-11-26T02:16:51.091265Z,2024-11-26T02:16:58.767123Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/27/S2...,"[{'Value': 'b8766bc6b8db7b60f6bf0dbfa514a825',...","{'Start': '2022-03-27T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.998768699998...,"{'type': 'Polygon', 'coordinates': [[[-60.9987..."
7,application/octet-stream,3d5aa2d4-142f-5ca8-93a3-b6bb85ad0603,S2B_MSIL2A_20220413T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-04-13T23:07:04.858000Z,2022-04-13T23:11:45.551651Z,2022-04-13T23:11:45.551651Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/04/13/S2B_MSIL...,[],"{'Start': '2022-04-13T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8

"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' and contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10) and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100"

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,28e6ae21-0310-4283-b1ed-efa8391301d3,S2B_MSIL2A_20220304T135109_N0510_R024_T20HPB_2...,application/octet-stream,856118643,2024-09-21T01:14:18.739000Z,2024-12-13T16:37:53.807406Z,2024-12-13T16:38:04.266684Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/04/S2...,"[{'Value': 'ab0092d5a2d32ff9f3fe7dd1bd598299',...","{'Start': '2022-03-04T13:51:09.000000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847718199 -3...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540000Z,2022-07-01T15:39:54.438282Z,2022-07-01T15:39:54.438282Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
2,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269000Z,2022-03-27T15:07:17.153349Z,2022-03-27T15:07:17.153349Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
3,application/octet-stream,c1be4316-de04-5902-b0ac-b01b32aec350,S2A_MSIL2A_20220312T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-12T23:14:08.829000Z,2022-03-26T01:57:48.785924Z,2022-03-26T01:57:48.785924Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/12/S2A_MSIL...,[],"{'Start': '2022-03-12T14:01:01.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.008057 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0080..."
4,application/octet-stream,41bd4637-3ca3-5ed3-bac7-6b5b66f6158e,S2A_MSIL2A_20220322T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-23T00:20:11.063000Z,2022-03-28T04:12:07.993475Z,2022-03-28T04:12:07.993475Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/22/S2A_MSIL...,[],"{'Start': '2022-03-22T14:01:01.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.002075 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0020..."
5,application/octet-stream,9ce5ef04-d1c3-5f9c-ae44-f1970d09e5e5,S2B_MSIL2A_20220327T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-27T19:15:06.268000Z,2022-03-27T19:20:05.514592Z,2022-03-27T19:20:05.514592Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/03/27/S2B_MSIL...,[],"{'Start': '2022-03-27T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.99878 -38.82...,"{'type': 'Polygon', 'coordinates': [[[-60.9987..."
6,application/octet-stream,d47474a5-b1b9-4824-bfb7-66b1ddfb91e1,S2B_MSIL2A_20220327T140049_N0510_R067_T20HPB_2...,application/octet-stream,528857225,2024-10-19T13:27:21.088000Z,2024-11-26T02:16:51.091265Z,2024-11-26T02:16:58.767123Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2022/03/27/S2...,"[{'Value': 'b8766bc6b8db7b60f6bf0dbfa514a825',...","{'Start': '2022-03-27T14:00:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-60.998768699998...,"{'type': 'Polygon', 'coordinates': [[[-60.9987..."
7,application/octet-stream,3d5aa2d4-142f-5ca8-93a3-b6bb85ad0603,S2B_MSIL2A_20220413T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-04-13T23:07:04.858000Z,2022-04-13T23:11:45.551651Z,2022-04-13T23:11:45.551651Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/04/13/S2B_MSIL...,[],"{'Start': '2022-04-13T13:51:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8

In [5]:
sys.path.append(r'../utils')
import mod_searcher as ms

In [6]:
# Salvo las búsquedas para utilizarla en notebook "Test_gdal.ipynb"
aux_file_pkl = '../Scripts/aux_files/search_s2.pkl'
ms.write_list(df_list,aux_file_pkl)

Done writing list into a binary file


In [7]:
# Salvo uno solo de los dataframes de búsqueda para que sea de esa
# manera que se busque y se baje los productos especificados.

path2df_search = '../Scripts/aux_files/search_s2_df.pkl'
display(df_list[0])
ms.write_df_search(df_list[0],path2df_search)
# ms.write_df(df,)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540Z,2022-07-01T15:39:54.438Z,2022-07-01T15:39:54.438Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269Z,2022-03-27T15:07:17.153Z,2022-03-27T15:07:17.153Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
2,application/octet-stream,3db1829f-7309-5c51-810f-20ffc7410946,S2A_MSIL2A_20220309T135121_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-09T21:38:02.833Z,2022-03-27T01:19:12.339Z,2022-03-27T01:19:12.339Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/09/S2A_MSIL...,[],"{'Start': '2022-03-09T13:51:21.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-60.554108 -39.6...,"{'type': 'Polygon', 'coordinates': [[[-60.5541..."


Done writing dataframe into a binary file
